In [1]:
import requests
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def get_page(url):
    soup = BeautifulSoup(requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text,'html.parser')
    if soup.find(class_='columnspecial'):
        return soup
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=webdriver.ChromeOptions())
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    
    return soup

def scrape_doctors(soup):
    if not soup: 
        return []
    
    doctors = []
    container = soup.find(id='placecontainer') or soup.find(id='colorlib-services') or soup
    
    for card in container.find_all(class_='columnspecial'):
        doctors.append({
            'name': card.find('p').get_text(strip=True) if card.find('p') else "N/A",
            'speciality': (card.find(class_='clinicdescription2') or card.find(class_='speciality')).get_text(strip=True) if card.find(class_='clinicdescription2') or card.find(class_='speciality') else "N/A",
            'link': card.find('a')['href'] if card.find('a') else "N/A"})

    return doctors

In [3]:
links = []
    
for i in range(25):
    url = f"https://u-medservices.com/specializationDetails/{i}"
    soup = get_page(url)
        
    if not soup:
        print("Failed to load page")
    else:
        doctors = scrape_doctors(soup)
        for i, doc in enumerate(doctors, 1):
            links.append(doc['link'])

In [4]:
names, new_links, add, specialty, phones, locations = [], [], [], [], [], []
counter = 1

for link in links:
    try:
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
        
        if "hollywoodstyle" in link:
            continue
            
        response = requests.get(link, headers=headers, timeout=10)
        
        if response.status_code != 200:
            print(f"Skipping URL {counter} (Status {response.status_code}): {link}")
            print("-" * 50)
            counter += 1
            continue
            
        soup = BeautifulSoup(response.text, 'html.parser')

        name = soup.find('h2', class_='title-category-details')
        name_text = name.get_text(strip=True) if name else "Not found"

        spec = soup.find('p', class_='clinicdescription')
        spec_text = spec.get_text(strip=True) if spec else "Not found"

        info_dict = {}
        for ul in soup.find_all('ul'):
            for li in ul.find_all('li'):
                for unwanted in li.find_all(['svg', 'script', 'style']):
                    unwanted.decompose()
                for comment in li.find_all(text=lambda t: isinstance(t, Comment)):
                    comment.extract()

                text = li.get_text(' ', strip=True)
                if ":" in text:
                    key, value = map(str.strip, text.split(":", 1))
                    info_dict[key] = value

        address = info_dict.get("العنوان", "Not found")
        phone = info_dict.get("التليفون", "Not found")
        location = "Not found"
        
        for li in soup.find_all('li'):
            has_map_icon = (li.find('svg', attrs={'data-icon': 'map-pin'}) or 
                          li.find('i', class_=lambda x: x and 'map-pin' in x))
            if has_map_icon:
                loc_text = li.get_text(' ', strip=True)
                loc_text = ' '.join(loc_text.split())
                if loc_text:
                    location = loc_text
                    break
        
        if location == "Not found":
            for span in soup.find_all('span', class_='dark'):
                span_text = span.get_text(strip=True)
                if span_text and len(span_text.split()) > 1:
                    location = span_text
                    break
        
        if location == "Not found":
            for icon in soup.find_all(['svg', 'i']):
                if 'map-pin' in str(icon.attrs):
                    next_sibling = icon.find_next_sibling(string=True)
                    if next_sibling and next_sibling.strip():
                        location = next_sibling.strip()
                        break

        print(f"URL: {counter}");          print(f"Name: {name_text}");      print(f"Specialty: {spec_text}")
        print(f"Address: {address}");      print(f"Location: {location}")
        print(f"Phone: {phone}");          print("-" * 50)
        
        names.append(name_text);           add.append(address);              new_links.append(link)
        specialty.append(spec_text);       phones.append(phone);             locations.append(location)
        
        counter += 1

    except Exception as e:
        print(f"Error processing URL {counter}: {str(e)}")
        counter += 1

C:\Users\ziad.saad\AppData\Local\Temp\ipykernel_6020\4004310434.py:32: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for comment in li.find_all(text=lambda t: isinstance(t, Comment)):


URL: 1
Name: أ.د/مها يوسف
Specialty: أستاذ مساعد طب الأطفال
Address: 19 شارع المشير أحمد إسماعيل
Location: سيدى جابر
Phone: 01210614225
--------------------------------------------------
URL: 2
Name: د/أدهم أحمد باديب
Specialty: مدرس واستشاري طب الأطفال وحديثي الولادة بكليه الطب جامعه الاسكندريه
Address: 22 شارع امين فكري
Location: محطه الرمل
Phone: 01270576997
--------------------------------------------------
URL: 3
Name: د/محمد سعيد ابوجبل
Specialty: استشاري طب الأطفالاستشاري أمراض الجهاز التنفسي والصدر والحساسية
Address: برج ميديكال تاور 8  ش كلية الطب ناصية شارع محمد رأفت الدور السادس
Location: محطة الرمل
Phone: 01094781134
--------------------------------------------------
URL: 4
Name: د/محمد ممدوح يوسف
Specialty: استشاري طب الأطفال وحديثي الولاده
Address: أعلى سوبر ماركت الشامي أمام استديو راضي - أبو يوسف
Location: العجمى البيطاش
Phone: 01275784025
--------------------------------------------------
URL: 5
Name: د/عاصم سليمان
Specialty: استشاري طب الأطفال
Address: الهانوفيل أمام 

In [ ]:
df = pd.DataFrame({"URL": new_links, 'Name': names , 'Speciality':specialty,  
                   'Location':locations , 'Address': add, 'Phone number': phones})

df.head(4)

In [142]:
df.to_csv('drs data.csv')